# EXTRACCION, LIMPIEZA Y CARGA DATA REFERENTE A DELITOS
VERSION 0.2

FECHA: 16/10/2020

ANALIZAR DELITOS VIGENTES Y NO VIGENTES


In [1]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
path_raw = "../data/raw/delitos"

In [3]:
codigos_delitos = pd.read_excel(f"{path_raw}/codigos_penal_2020.xlsx", sheet_name = "codigos vigentes")

In [4]:
codigos_delitos = codigos_delitos.drop_duplicates() # elimino filas con NaN
codigos_delitos = codigos_delitos.drop([0,1,2], axis = 0) # elimino 2 primeras filas que son titulos

In [5]:
# elimino columnas con datos NaN

variables = range(2,248)
columnas = []
for variable in variables:
    columnas.append("Unnamed: " + str(variable))

codigos_delitos = codigos_delitos.drop(columns = columnas, axis = 1)

In [6]:
# cambio nombres columnas

codigos_delitos = codigos_delitos.rename(columns = {'VERSION AL 01/01/2018':'COD. MATERIA', 'Unnamed: 1':'MATERIA'})

In [7]:
codigos_delitos

,COD. MATERIA,MATERIA
5,LIBRO I TITULO IV QUEBRANTAMIENTO DE SENTEN...,NaN
7,101,- QUEBRANTAMIENTO. ART. 90
8,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NaN
10,202,- SECUESTRO. ART. 141
11,203,- SUSTRACCIÓN DE MENORES. ART. 142
...,...,...
507,532,Interrumpir libre circulación (Ley 21208 inc 1))
508,533,Lanzar obj a vía púb con muerte o lesiones (Le...
509,870,Robo en ocasión de calamidad o alteración al o...
510,871,Hurto en ocasión de calamidad o alteración al ...


In [8]:
delitos_vigentes = []

for item in codigos_delitos.index:

    if str(codigos_delitos['COD. MATERIA'][item]).isupper():
        tipologia_delito=str(codigos_delitos['COD. MATERIA'][item])
    else:
        delitos_vigentes.append([codigos_delitos['COD. MATERIA'][item],
                                str(codigos_delitos['MATERIA'][item]).upper().rstrip(),
                                tipologia_delito,'VIGENTE'])
        

In [9]:
df_delitos_vigentes = pd.DataFrame(delitos_vigentes,columns = ['COD. MATERIA','MATERIA','TIPOLOGIA MATERIA','VIGENCIA MATERIA'])

In [10]:
df_delitos_vigentes

,COD. MATERIA,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA
0,101,- QUEBRANTAMIENTO. ART. 90,LIBRO I TITULO IV QUEBRANTAMIENTO DE SENTEN...,VIGENTE
1,202,- SECUESTRO. ART. 141,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
2,203,- SUSTRACCIÓN DE MENORES. ART. 142,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
3,204,- VIOLACIÓN DE MORADA. ART.144,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
4,205,"- APERTURA , REGISTRO O INTERCEPTACIÓN DE CORR...",LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
...,...,...,...,...
435,532,INTERRUMPIR LIBRE CIRCULACIÓN (LEY 21208 INC 1)),LEY 21.208,VIGENTE
436,533,LANZAR OBJ A VÍA PÚB CON MUERTE O LESIONES (LE...,LEY 21.208,VIGENTE
437,870,ROBO EN OCASIÓN DE CALAMIDAD O ALTERACIÓN AL O...,LEY 21.208,VIGENTE
438,871,HURTO EN OCASIÓN DE CALAMIDAD O ALTERACIÓN AL ...,LEY 21.208,VIGENTE


In [11]:
df_delitos_vigentes.dtypes

COD. MATERIA         object
MATERIA              object
TIPOLOGIA MATERIA    object
VIGENCIA MATERIA     object
dtype: object

## Limpieza de variable MATERIA

In [12]:
# Elimino tildes de las columnas object

cols = df_delitos_vigentes.select_dtypes(include = ["object"]).columns
df_delitos_vigentes[cols] = df_delitos_vigentes[cols].progress_apply(cleandata.elimina_tilde)
df_delitos_vigentes[cols] = df_delitos_vigentes[cols].progress_apply(cleandata.limpieza_caracteres)

100%|██████████| 4/4 [00:00<00:00, 353.79it/s]


In [13]:
df_delitos_vigentes['COD. MATERIA'] = df_delitos_vigentes['COD. MATERIA'].fillna(0).astype('int16')

## CARGA Y LIMPIEZA DE DATOS RELACIONADOS A DELITOS NO VIGENTES


In [14]:
codigos_delitos_novigentes = pd.read_excel(f"{path_raw}/codigos_penal_2020.xlsx", sheet_name = "Codigos no vigentes")

In [15]:
# cambio nombres columnas

codigos_delitos_novigentes = codigos_delitos_novigentes.rename(columns = {'MATERIAS PENALES NO VIGENTES':'TIPOLOGIA MATERIA',
                                              'Unnamed: 1':'COD. MATERIA','Unnamed: 2':'MATERIA'})

In [16]:
codigos_delitos_novigentes = codigos_delitos_novigentes.drop([0], axis = 0) # elimino primera fila que son titulos
codigos_delitos_novigentes = codigos_delitos_novigentes.fillna('ST') # reemplazo Nan por ST

In [17]:
codigos_delitos_novigentes

,TIPOLOGIA MATERIA,COD. MATERIA,MATERIA
1,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,201,LIBERTAD DE CULTO. ...
2,ST,206,"EXACCIONES ILEGALES (ART. 147, 157 Y 241 CÓDIG..."
3,ST,208,VIOLACION DE PRERROGATIVAS. ...
4,ST,209,CONDENAS IRREGULARES. ...
5,ST,211,DELITO CONTRA LIBERTAD DE OPINION. ...
...,...,...,...
98,INFRACCION A OTROS TEXTOS LEGALES,12050,INFRACCION ORDENANZA ADUANAS (FRAUDE Y C...
99,ST,12154,DAÑOS O APROPIACION SOBRE MONUMENTOS NACIONALE...
100,ST,13018,"VIOLENCIA EN LOS ESTADIOS ART. 6, 6A,6B,6D,6E,..."
101,ST,17001,DELITOS QUE COMPROMETEN RELACIONES INTERNACION...


In [18]:
delitos_no_vigentes = []
for item in codigos_delitos_novigentes.index:
    
    tipologia_delito = codigos_delitos_novigentes['TIPOLOGIA MATERIA'][item]
    
    if tipologia_delito != 'ST':
        tipologia = codigos_delitos_novigentes['TIPOLOGIA MATERIA'][item]
    else:
        tipologia_delito = tipologia
    
    delitos_no_vigentes.append([codigos_delitos_novigentes['COD. MATERIA'][item],
                                  codigos_delitos_novigentes['MATERIA'][item].rstrip(),
                                  tipologia_delito,'NO VIGENTE'])
        

In [19]:
df_delitos_no_vigentes = pd.DataFrame(delitos_no_vigentes, columns = ['COD. MATERIA','MATERIA','TIPOLOGIA MATERIA','VIGENCIA MATERIA'])

In [20]:
df_delitos_no_vigentes

,COD. MATERIA,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA
0,201,LIBERTAD DE CULTO.,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NO VIGENTE
1,206,"EXACCIONES ILEGALES (ART. 147, 157 Y 241 CÓDIG...",LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NO VIGENTE
2,208,VIOLACION DE PRERROGATIVAS.,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NO VIGENTE
3,209,CONDENAS IRREGULARES.,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NO VIGENTE
4,211,DELITO CONTRA LIBERTAD DE OPINION.,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NO VIGENTE
...,...,...,...,...
97,12050,INFRACCION ORDENANZA ADUANAS (FRAUDE Y C...,INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE
98,12154,DAÑOS O APROPIACION SOBRE MONUMENTOS NACIONALE...,INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE
99,13018,"VIOLENCIA EN LOS ESTADIOS ART. 6, 6A,6B,6D,6E,...",INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE
100,17001,DELITOS QUE COMPROMETEN RELACIONES INTERNACION...,INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE


In [21]:
# Elimino tildes de las columnas object

cols = df_delitos_no_vigentes.select_dtypes(include = ["object"]).columns
df_delitos_no_vigentes[cols] = df_delitos_no_vigentes[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 3/3 [00:00<00:00, 322.76it/s]


In [22]:
df_delitos_no_vigentes['COD. MATERIA'] = df_delitos_no_vigentes['COD. MATERIA'].astype('int16')

# UNION DE AMBOS DATASET CON DELITOS VIGENTES Y NO VIGENTES

In [23]:
df_delitos = pd.concat([df_delitos_vigentes,df_delitos_no_vigentes])

In [24]:
df_delitos

,COD. MATERIA,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA
0,101,QUEBRANTAMIENTO. ART. 90,LIBRO I TITULO IV QUEBRANTAMIENTO DE SENTEN...,VIGENTE
1,202,SECUESTRO. ART. 141,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
2,203,SUSTRACCION DE MENORES. ART. 142,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
3,204,VIOLACION DE MORADA. ART.144,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
4,205,"APERTURA , REGISTRO O INTERCEPTACION DE CORRE...",LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
...,...,...,...,...
97,12050,INFRACCION ORDENANZA ADUANAS (FRAUDE Y C...,INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE
98,12154,DAÑOS O APROPIACION SOBRE MONUMENTOS NACIONALE...,INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE
99,13018,"VIOLENCIA EN LOS ESTADIOS ART. 6, 6A,6B,6D,6E,...",INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE
100,17001,DELITOS QUE COMPROMETEN RELACIONES INTERNACION...,INFRACCION A OTROS TEXTOS LEGALES,NO VIGENTE


In [25]:
df_delitos.sort_values("COD. MATERIA")

,COD. MATERIA,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA
0,101,QUEBRANTAMIENTO. ART. 90,LIBRO I TITULO IV QUEBRANTAMIENTO DE SENTEN...,VIGENTE
0,201,LIBERTAD DE CULTO.,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,NO VIGENTE
1,202,SECUESTRO. ART. 141,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
2,203,SUSTRACCION DE MENORES. ART. 142,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
3,204,VIOLACION DE MORADA. ART.144,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
...,...,...,...,...
381,23100,"INFRACCIONES A LA LEY 19.496, SOBRE PROTECCIO...",INFRACCION A OTROS TEXTOS LEGALES,VIGENTE
382,23200,"INFRACCIONES A LA LEY 19132, SOBRE TVN ART 9 ...",INFRACCION A OTROS TEXTOS LEGALES,VIGENTE
383,23300,INFRACCIONES A LA LEY 18.287 SOBRE JUZGADO DE...,INFRACCION A OTROS TEXTOS LEGALES,VIGENTE
384,23400,INFRACCIONES A LA LEY 19220 SOBRE ESTABLECIM...,INFRACCION A OTROS TEXTOS LEGALES,VIGENTE


## Guardamos DF como to_feather

In [27]:
# Guardamos dataset como archivo feather
df_delitos.reset_index(inplace = True)
path_processed = "../data/processed/delitos"
os.makedirs(path_processed, exist_ok = True) 

df_delitos.to_feather(f'{path_processed}/Delitos_feather')